In [2]:
import numpy as np
import pandas as pd
import yfinance as yf

ModuleNotFoundError: No module named 'yfinance'

## Step 1 
- pd.read_html() returns a list of tables (converted to dataframes) from an HTML page
- Grab the first table using 0-index & select the columns we want

In [ ]:

tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0][['Symbol', 'GICS Sector']]
tickers

## Step 2
Clean the ticker data. Yahoo Finance has tickers for some companies with a hyphen in them. Ex. Brown-Forman Corporation (NYSE: BF-B). However, the ticker from the Wikipedia page is in the form BF.B. Yfinance will throw an error so clean the data to prevent errors.

In [ ]:
# Replace all '.' with a '-' in the symbol column
type(tickers['Symbol'])

# Internally, Pandas columns are stored as Series objects, which are essentially NumPy arrays (with some extra methods and properties)
# This is why we can use NumPy's vectorized string methods on the column at once rather than looping through each row
tickers['Symbol'] = tickers['Symbol'].str.replace('.', '-')
tickers[tickers['Symbol'] == 'BF-B']

## Step 3
- Convert the dataframe to a dictionary of key-value pairs
- Keys: Sectors, Values: Tickers in each sector

In [ ]:
# Breakdown dataframe into keys of sectors & values of tickers
sector_breakdown = tickers.groupby('GICS Sector')['Symbol'].apply(list)
sector_breakdown = sector_breakdown.to_dict()
sector_breakdown

## Step 4
Get all tickers into a list to put into yfinance

In [ ]:
# Get all tickers from sector_breakdown into a single list
ticker_list = []
for sector in sector_breakdown:
    ticker_list.extend(sector_breakdown[sector])

ticker_list   

## Step 5
- Extract data for all tickers through yfinance
- Extract the Adj Close for the past month

In [ ]:
# Get today's date
today = pd.Timestamp.today().strftime('%Y-%m-%d')

# Get the date 1 month ago
month_ago = pd.Timestamp.today() - pd.DateOffset(months=1)

# Get data from yfinance for all tickers
data = yf.download(ticker_list, start=month_ago, end=today)

# Keep just the Adj Close column
data = data.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'])
data

# IMPORTANT: Notice how yfinance doesn't throw an error for tickers like BF.B since we cleaned the inputs before extracting our dataset

## Step 6
We need to do some additional data cleaning. For some days, Yahoo Finance may not have stock price values. Those values will be marked as NaN, or Null. Null indicates the absence of values. We can't have NaN values because our calculations later on will throw errors.

To fix this, we are going to use the **backward fill** followed by the **forward fill** method. These method basically takes the value from the previous index (for forward fill) or the next index (for backward fill) and fills the NaN index with it. We are going to use an **axis value of 0**, which means we take the value from the previous (or next) row to fill it in.

![Alt text](image.png)

In [ ]:
# Backward fill NaN values
data = data.fillna(method='bfill', axis=0)

# Backward fill NaN values
data = data.fillna(method='ffill', axis=0)

# Print number of NA values across each column
print(data.isna().sum(axis=1))

# If you have NaN values still, then account for this when making calculations further down the line
# Using forward fill means we account for situations like this: 3.0 3.0 NaN NaN NaN NaN
# Using backward fill means we account for situations like this: NaN NaN NaN NaN 3.0 3.0

## Step 7
- Calculate our daily returns using the **pct_change()** function in Pandas
- Aggregate our returns by sector
- Calculate average daily return for each sector

In [ ]:
# Calculate daily returns
returns = data.pct_change()

# Drop the index name Adj Close
returns = returns.droplevel(0, axis=1)

# Group tickers in returns dataframe by sector using sector_breakdown's keys & values
sector_mean_returns = {}

# Extract each key & value pair from sector_breakdown
for sector, tickers in sector_breakdown.items():
    # Get tickers for each sector
    sector_data = returns[tickers]

    # Calculate mean returns for each sector and add to dictionary
    sector_mean_returns[sector] = sector_data.mean(axis=1)

# Convert sector_mean_returns dict to a DataFrame
sector_mean_returns = pd.DataFrame(sector_mean_returns)

sector_mean_returns

## Step 8
Drop rows that have NaN values

In [ ]:
# Drop rows that have NaN values
sector_mean_returns = sector_mean_returns.dropna()
sector_mean_returns

## Step 9
Define a function that does the following:
- Calculate overall portfolio return
- Calculate overall portfolio volatility
- Calculate Sharpe Ratio

Return a list with all 3 statistics

In [ ]:
def portfolio_stats(weights, sector_mean_returns):

    # Since we have data for 1 month, multiply by 12 to get annualized return
    annualization_factor = 12

    # Calculate portfolio return using the annualization factor
    portfolio_return_annual = np.sum(sector_mean_returns.mean(axis=0) * weights) * annualization_factor

    # Calculate portfolio volatility (standard deviation)
    portfolio_volatility_monthly = np.sqrt(np.dot(weights.T, np.dot(sector_mean_returns.cov(), weights)))

    # Annualize the volatility by multiplying by the square root of 12
    portfolio_volatility_annual = portfolio_volatility_monthly * np.sqrt(annualization_factor)

    # Calculate portfolio sharpe ratio
    sharpe_ratio = portfolio_return_annual / portfolio_volatility_annual

    return np.array([portfolio_return_annual, portfolio_volatility_annual, sharpe_ratio])

# Test the function with sample weights for all 11 S&P 500 sectors
weights = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05, 0.05])
output = pd.DataFrame(portfolio_stats(weights, sector_mean_returns), index=['Annual Portfolio Return', 'Annual Portfolio Volatility', 'Sharpe Ratio'], columns=['Value'])
output

## Step 9 (Cont.)
Visualizing the variance-covariance matrix (not necessary, just showing you what happens so you understand)

In [ ]:
# Calculate the variance-covariance matrix for the 11 sector returns
cov_matrix = sector_mean_returns.cov()
cov_matrix